In [1]:
import pandas as pd
import csv
import statsmodels.api as sm

from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/sf_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
df_listings = pd.read_csv(listings_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

print (df_listings.shape)
df_listings.head()

(6633, 96)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,2.018080e+13,2018-08-06,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,none,*Quiet cul de sac in friendly neighborhood *St...,...,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,5858,https://www.airbnb.com/rooms/5858,2.018080e+13,2018-08-06,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,7918,https://www.airbnb.com/rooms/7918,2.018080e+13,2018-08-06,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,none,"Shopping old town, restaurants, McDonald, Whol...",...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16
3,8142,https://www.airbnb.com/rooms/8142,2.018080e+13,2018-08-06,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,none,NaN,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.15
4,8339,https://www.airbnb.com/rooms/8339,2.018080e+13,2018-08-06,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,none,NaN,...,t,STR-0000264,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,t,t,2,0.24


In [4]:
df_listings.isnull().sum()

id                                     0
listing_url                            0
scrape_id                              0
last_scraped                           0
name                                   0
summary                              228
space                               1041
description                           24
experiences_offered                    0
neighborhood_overview               1917
notes                               2587
transit                             1851
access                              2057
interaction                         2182
house_rules                         1778
thumbnail_url                       6633
medium_url                          6633
picture_url                            0
xl_picture_url                      6633
host_id                                0
host_url                               0
host_name                              0
host_since                             0
host_location                          7
host_about      

In [5]:
df_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [6]:
# Noted >90% null value in 'experience_offered', 'host_acceptance_rate', 'neighbourhood_group_cleansed'
# removed 'neighbourhood',  kept'neighbourhood_cleansed',
# both city and 'smart location' provides city level information, and 'smart_location can be dropped
# country code and country can be dropped, as all data located within US
columns_to_drop =['listing_url', 'scrape_id', 'last_scraped', 'experiences_offered', 
                  'thumbnail_url', 'medium_url', 'xl_picture_url', 'host_acceptance_rate','street',
                  'neighbourhood', 
       'neighbourhood_group_cleansed','zipcode','smart_location','country_code', 
                  'country', 'square_feet','weekly_price', 'monthly_price','calendar_updated', 
                  'calendar_last_scraped', 'requires_license', 'license']

In [7]:
# 'host_since', 'host_location',

host_related_info_to_drop = ['host_id', 'host_url', 'host_name', 'host_since',
                             'host_location', 'host_about', 'host_thumbnail_url',
                             'host_picture_url', 'host_neighbourhood']

In [8]:
# The information is useful for now, but can be dropped during modeling

extra_info_to_drop = []

In [9]:
df_smaller_listings = df_listings.drop (columns_to_drop, axis=1)

In [10]:
df_smaller_listings

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,...,review_scores_location,review_scores_value,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,958,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,...,10.0,10.0,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,5858,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,I love how our neighborhood feels quiet but is...,All the furniture in the house was handmade so...,The train is two blocks away and you can stop ...,"Our deck, garden, gourmet kitchen and extensiv...",NaN,...,10.0,9.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,7918,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,"Shopping old town, restaurants, McDonald, Whol...",Please email your picture id with print name (...,N Juda Muni and bus stop. Street parking.,NaN,NaN,...,9.0,8.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16
3,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,NaN,Please email your picture id with print name (...,"N Juda Muni, Bus and UCSF Shuttle. small shopp...",NaN,NaN,...,9.0,9.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.15
4,8339,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,NaN,tax ID on file tax ID on file,NaN,Guests have access to everything listed and sh...,NaN,...,10.0,10.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,t,t,2,0.24
5,8567,Lovely 2BR flat Great Location,NaN,"Fully furnished 2BR, 1BA flat in beautiful Vic...","Fully furnished 2BR, 1BA flat in beautiful Vic...","The neighborhood is very centrally located, cl...",NaN,We're 2 blocks from several bus lines that can...,NaN,"I am generally here to welcome you, and if not...",...,9.0,9.0,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,0.27
6,8739,"Mission Sunshine, with Private Bath","Welcome to San Francisco's Mission District, t...","Your sunny room has a queen size bed, and look...","Welcome to San Francisco's Mission District, t...",Located between Valencia Street and Dolores Pa...,"We live in a dense, urban neighborhood, and ou...",It's an easy and lovely 10 minute stroll to th...,Guests have their own bedroom and private bath...,"We are experienced Airbnb hosts, and have had ...",...,10.0,9.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,2,5.41
7,9225,Artful Potrero Separate Floor with Garden,A Unique Guest Suite! A Spacious Art Filled ...,"It is unique, very spacious [800 sq. ft. with...",A Unique Guest Suite! A Spacious Art Filled ...,"It is a distinct neighborhood, with easy acces...",NaN,There are buses within a block to connect to a...,"Laundry privileges, Spacious Garden with City ...",We are available to make suggestions to enhanc...,...,10.0,9.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,3.84
8,10251,Victorian Suite in Inner Mission,NaN,Please read this before you book! Second floor...,Please read this before you book! Second floor...,"Neighborhood is safe, sunny, lively and fun

In [11]:
# This is to onvert strings with percentage sign to float

def p2f(string):
    """
    Convert strings with percentage sign to float
    """
    percentage = float(str(string).strip('%'))/100
    return percentage

In [12]:
# Convert host_response_rate to float

df_smaller_listings['host_response_rate'] = df_smaller_listings['host_response_rate'].apply(lambda x: p2f(x))

In [13]:
test = df_smaller_listings['price'][0]

In [14]:
def c2f(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$'))
    return float_

In [15]:
c2f(test)

170.0

In [16]:
# Loaded reviews data. Not used at this moement

In [17]:
df_reviews = pd.read_csv(reviews_file)

print (df_reviews.shape)
df_reviews.head()

(278884, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...


In [18]:
df_calendar =pd.read_csv(calendar_file)

print (df_calendar.shape)
df_calendar.head()

(2420680, 4)


,listing_id,date,available,price
0,958,2019-08-05,f,NaN
1,958,2019-08-04,f,NaN
2,958,2019-08-03,f,NaN
3,958,2019-08-02,f,NaN
4,958,2019-08-01,f,NaN


In [19]:
df_smaller_listings.columns

Index(['id', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'picture_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'state', 'market', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_36

In [20]:
df_smaller_listings.head()

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,...,review_scores_location,review_scores_value,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,958,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,...,10.0,10.0,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,5858,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,I love how our neighborhood feels quiet but is...,All the furniture in the house was handmade so...,The train is two blocks away and you can stop ...,"Our deck, garden, gourmet kitchen and extensiv...",NaN,...,10.0,9.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,7918,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,"Shopping old town, restaurants, McDonald, Whol...",Please email your picture id with print name (...,N Juda Muni and bus stop. Street parking.,NaN,NaN,...,9.0,8.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16
3,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,NaN,Please email your picture id with print name (...,"N Juda Muni, Bus and UCSF Shuttle. small shopp...",NaN,NaN,...,9.0,9.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.15
4,8339,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,NaN,tax ID on file tax ID on file,NaN,Guests have access to everything listed and sh...,NaN,...,10.0,10.0,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,t,t,2,0.24


In [21]:
drop_for_simple_model=['id', 'name', 'summary', 'space', 'description', 
                       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
                       'house_rules', 'picture_url', 'host_id', 'host_url', 'host_name','host_about',
                      'host_thumbnail_url', 'host_picture_url', 'latitude', 'longitude',]

In [22]:
df_simple_model = df_smaller_listings.drop(drop_for_simple_model, axis =1)

In [23]:
df_simple_model.columns

Index(['host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'state', 'market',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'jurisdiction_names', 'ins

In [24]:
# df_simple_model['host_since']   = pd.to_datetime(df_smaller_listings['host_since'])
# df_simple_model['first_review'] = pd.to_datetime(df_smaller_listings['first_review'])
# df_simple_model['last_review']  = pd.to_datetime(df_smaller_listings['last_review'])

In [25]:
def duration(starting_time):
    """
    Calculate duration or number of years between current year and input years
    """
    starting_year = starting_time.year
    current_year = datetime.now().year
    duration = current_year - starting_year + 1
    
    return duration

In [26]:
# Calculate durations for timestamp columns

df_simple_model['host_duration'] = df_simple_model['host_since'].apply(lambda x: duration(x))
df_simple_model['yrs_since_first_review'] = df_simple_model['first_review'].apply(lambda x: duration(x))
df_simple_model['yrs_since_last_review'] = df_simple_model['last_review'].apply(lambda x: duration(x))

In [27]:
time_stamps =['host_since', 'first_review', 'last_review']
df_simple_model.drop(time_stamps, axis=1, inplace=True)

In [28]:
# Replace special characters in columns

df_simple_model.host_verifications = df_simple_model.host_verifications.str.replace('[^\w\s]', '')
df_simple_model.jurisdiction_names = df_simple_model.jurisdiction_names.str.replace('[^\w\s]', '')
df_simple_model.amenities = df_simple_model.amenities.str.replace('[^\w\s]', '')
df_simple_model.security_deposit = df_simple_model.security_deposit.str.replace('[^\w\s]', '')
df_simple_model.cleaning_fee = df_simple_model.cleaning_fee.str.replace('[^\w\s]', '')
df_simple_model.price = df_simple_model.price.str.replace('[^\w\s]', '')

In [29]:
df_simple_model.shape

(6633, 54)

In [30]:
df_simple_model.columns

Index(['host_location', 'host_response_time', 'host_response_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'state', 'market',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'jurisdiction_names',
       'instant_bookable', 'is_business_travel_ready', '

In [31]:
df_simple_model.isnull().sum()

host_location                          7
host_response_time                   575
host_response_rate                   575
host_is_superhost                      0
host_neighbourhood                   374
host_listings_count                    0
host_total_listings_count              0
host_verifications                     0
host_has_profile_pic                   0
host_identity_verified                 0
neighbourhood_cleansed                 0
city                                  12
state                                  2
market                                20
is_location_exact                      0
property_type                          0
room_type                              0
accommodates                           0
bathrooms                             26
bedrooms                               1
beds                                   5
bed_type                               0
amenities                              0
price                                  0
security_deposit

In [32]:
# Drop null values for a few columns

null_values_to_drop = ['host_location','host_response_rate','host_neighbourhood', 
                       'state', 'market', 'bathrooms', 'bedrooms','review_scores_rating', 
                       'review_scores_value', 'jurisdiction_names']
df_simple_model_cleaned = df_simple_model.dropna(subset=null_values_to_drop)

In [33]:
df_simple_model_cleaned.shape

(4685, 54)

In [34]:
df_simple_model_cleaned.isnull().sum()

host_location                         0
host_response_time                    0
host_response_rate                    0
host_is_superhost                     0
host_neighbourhood                    0
host_listings_count                   0
host_total_listings_count             0
host_verifications                    0
host_has_profile_pic                  0
host_identity_verified                0
neighbourhood_cleansed                0
city                                  0
state                                 0
market                                0
is_location_exact                     0
property_type                         0
room_type                             0
accommodates                          0
bathrooms                             0
bedrooms                              0
beds                                  0
bed_type                              0
amenities                             0
price                                 0
security_deposit                    885


In [35]:
# Convert column to numeric value. Prepare for filling null values with average

df_simple_model_cleaned['security_deposit'] = df_simple_model_cleaned['security_deposit']  \
                                              .apply(pd.to_numeric, errors='coerce')

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
df_simple_model_cleaned.security_deposit

0        10000.0
1            NaN
2        20000.0
3        20000.0
4            0.0
5            0.0
6            0.0
7            NaN
8        50000.0
9        50000.0
10      300000.0
11      250000.0
12      300000.0
13      250000.0
14       20000.0
15       20000.0
16           NaN
17       89900.0
18           NaN
19      500000.0
20           NaN
21       13000.0
22       33300.0
23       50000.0
24       20000.0
25       50000.0
26       50000.0
27       20000.0
28       50000.0
29       25000.0
          ...   
6372         NaN
6373     50000.0
6376         0.0
6377         0.0
6381     30000.0
6382     30000.0
6406         0.0
6407         0.0
6422         NaN
6425         0.0
6426     30000.0
6448         0.0
6449     40000.0
6453         0.0
6454         0.0
6455     35000.0
6464     80000.0
6466     50000.0
6468     20000.0
6469     30000.0
6484     25000.0
6485         0.0
6488     10000.0
6491     40000.0
6495         0.0
6507         NaN
6540     25000.0
6542         N

In [37]:
# Filled null values with average value

df_simple_model_cleaned['security_deposit'].fillna(
    value=df_simple_model_cleaned['security_deposit'].mean(),
    inplace=True)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [38]:
df_simple_model_cleaned['security_deposit'] = (df_simple_model_cleaned['security_deposit']>0).astype(int)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_simple_model_cleaned['price'] = df_simple_model_cleaned['price']  \
                                   .apply(pd.to_numeric, errors='coerce')

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_simple_model_cleaned.security_deposit

0       1
1       1
2       1
3       1
4       0
5       0
6       0
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
       ..
6372    1
6373    1
6376    0
6377    0
6381    1
6382    1
6406    0
6407    0
6422    1
6425    0
6426    1
6448    0
6449    1
6453    0
6454    0
6455    1
6464    1
6466    1
6468    1
6469    1
6484    1
6485    0
6488    1
6491    1
6495    0
6507    1
6540    1
6542    1
6564    1
6615    0
Name: security_deposit, Length: 4685, dtype: int64

In [41]:
# Convert column to numeric value. Prepare for filling null values with average

df_simple_model_cleaned['cleaning_fee'] = df_simple_model_cleaned['cleaning_fee']  \
                                              .apply(pd.to_numeric, errors='coerce')


/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
# Filled null values with average value

df_simple_model_cleaned['cleaning_fee'].fillna(
    value=df_simple_model_cleaned['cleaning_fee'].mean(),
    inplace=True)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [43]:
df_simple_model_cleaned['cleaning_fee'] = (df_simple_model_cleaned['cleaning_fee']>0).astype(int)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
df_simple_model_cleaned['cleaning_fee']

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
       ..
6372    1
6373    1
6376    1
6377    1
6381    1
6382    1
6406    1
6407    1
6422    1
6425    1
6426    1
6448    1
6449    1
6453    1
6454    1
6455    1
6464    1
6466    1
6468    1
6469    1
6484    1
6485    1
6488    1
6491    1
6495    1
6507    1
6540    1
6542    1
6564    1
6615    1
Name: cleaning_fee, Length: 4685, dtype: int64

In [45]:
df_simple_model_cleaned.isnull().sum()

host_location                       0
host_response_time                  0
host_response_rate                  0
host_is_superhost                   0
host_neighbourhood                  0
host_listings_count                 0
host_total_listings_count           0
host_verifications                  0
host_has_profile_pic                0
host_identity_verified              0
neighbourhood_cleansed              0
city                                0
state                               0
market                              0
is_location_exact                   0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                           0
bedrooms                            0
beds                                0
bed_type                            0
amenities                           0
price                               0
security_deposit                    0
cleaning_fee                        0
guests_inclu

In [46]:
df_simple_model_cleaned.head()

,host_location,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,host_duration,yrs_since_first_review,yrs_since_last_review
0,"San Francisco, California, United States",within an hour,0.92,t,Duboce Triangle,1,1,email phone facebook reviews kba,t,t,...,f,f,moderate,f,f,1,1.38,11,10.0,1.0
1,"San Francisco, California, United States",within an hour,1.00,f,Bernal Heights,2,2,email phone reviews kba work_email,t,t,...,f,f,strict_14_with_grace_period,f,f,1,0.99,10,10.0,2.0
2,"San Francisco, California, United States",within a few hours,1.00,f,Cole Valley,10,10,email phone reviews jumio government_id,t,t,...,f,f,strict_14_with_grace_period,f,f,9,0.16,10,10.0,3.0
3,"San Francisco, California, United States",within a few hours,1.00,f,Cole Valley,10,10,email phone reviews jumio government_id,t,t,...,f,f,strict_14_with_grace_period,f,f,9,0.15,10,5.0,2.0
4,"San Francisco, California, United States",within an hour,1.00,f,Alamo Square,2,2,email phone reviews kba,t,t,...,f,f,strict_14_with_grace_period,t,t,2,0.24,10,10.0,2.0


In [47]:
df_simple_model_cleaned.columns

Index(['host_location', 'host_response_time', 'host_response_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'state', 'market',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'jurisdiction_names',
       'instant_bookable', 'is_business_travel_ready', '

In [48]:
################## drop for simplicity purpose

df_test_model = df_simple_model_cleaned.drop(['amenities', 'host_verifications'], axis=1)

In [49]:
df_test_model.shape

(4685, 52)

In [50]:
# 'host_verifications','amenities',

categorical_data = ['host_location', 'host_response_time', 
       'host_is_superhost', 'host_neighbourhood', 
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'state', 'market',
       'is_location_exact', 'property_type', 'room_type',  'bed_type', 'has_availability',
        'jurisdiction_names',
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       ]

In [51]:
df_test_model = pd.get_dummies(df_test_model, prefix = categorical_data, columns = categorical_data )

In [52]:
df_test_model.columns

Index(['host_response_rate', 'host_listings_count',
       'host_total_listings_count', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'security_deposit', 'cleaning_fee',
       ...
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict',
       'cancellation_policy_strict_14_with_grace_period',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60',
       'require_guest_profile_picture_f', 'require_guest_profile_picture_t',
       'require_guest_phone_verification_f',
       'require_guest_phone_verification_t'],
      dtype='object', length=346)

In [53]:
df_test_model.isnull().sum()

host_response_rate                                 0
host_listings_count                                0
host_total_listings_count                          0
accommodates                                       0
bathrooms                                          0
bedrooms                                           0
beds                                               0
price                                              0
security_deposit                                   0
cleaning_fee                                       0
guests_included                                    0
extra_people                                       0
minimum_nights                                     0
maximum_nights                                     0
availability_30                                    0
availability_60                                    0
availability_90                                    0
availability_365                                   0
number_of_reviews                             

In [54]:
df_test_model['price'] = df_test_model.price.apply(lambda x: x/100)

In [55]:
df_test_model.price

0       170.0
1       235.0
2        65.0
3        65.0
4       675.0
5       255.0
6       139.0
7       135.0
8       265.0
9       120.0
10      218.0
11      177.0
12      194.0
13      139.0
14       85.0
15       85.0
16       79.0
17      135.0
18      215.0
19      450.0
20      107.0
21      110.0
22      198.0
23      150.0
24       65.0
25      225.0
26       95.0
27      155.0
28      178.0
29      199.0
        ...  
6372     70.0
6373    275.0
6376    200.0
6377    166.0
6381     99.0
6382     89.0
6406    108.0
6407    180.0
6422    185.0
6425    334.0
6426    175.0
6448     89.0
6449    150.0
6453     90.0
6454     90.0
6455    499.0
6464    499.0
6466    250.0
6468    150.0
6469    160.0
6484    150.0
6485    129.0
6488    217.0
6491    300.0
6495    450.0
6507     95.0
6540    256.0
6542     45.0
6564    179.0
6615     74.0
Name: price, Length: 4685, dtype: float64

In [56]:
df_test_model.columns

Index(['host_response_rate', 'host_listings_count',
       'host_total_listings_count', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'security_deposit', 'cleaning_fee',
       ...
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict',
       'cancellation_policy_strict_14_with_grace_period',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60',
       'require_guest_profile_picture_f', 'require_guest_profile_picture_t',
       'require_guest_phone_verification_f',
       'require_guest_phone_verification_t'],
      dtype='object', length=346)

In [57]:
# Create correlation matrix
corr_matrix = df_test_model.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

In [58]:
to_drop

['host_total_listings_count',
 'availability_90',
 'host_is_superhost_t',
 'host_neighbourhood_Amed',
 'host_neighbourhood_Belmont',
 'host_neighbourhood_El Grenada',
 'host_neighbourhood_Gay Village',
 'host_neighbourhood_Lincoln Park',
 'host_neighbourhood_Little Havana',
 'host_neighbourhood_Malibu',
 'host_neighbourhood_Midtown East',
 'host_neighbourhood_Napili/Honokowai',
 'host_neighbourhood_Waikiki',
 'host_has_profile_pic_t',
 'host_identity_verified_t',
 'neighbourhood_cleansed_Bayview',
 'neighbourhood_cleansed_Bernal Heights',
 'neighbourhood_cleansed_Excelsior',
 'neighbourhood_cleansed_Presidio',
 'is_location_exact_t',
 'room_type_Private room',
 'instant_bookable_t',
 'require_guest_profile_picture_t',
 'require_guest_phone_verification_t']

In [59]:
# Drop features 
df_test_model.drop(to_drop, axis=1, inplace=True)

In [60]:
X = df_test_model.drop(['price', 'host_response_rate', 'extra_people',  'cleaning_fee'], axis=1)
y = df_test_model['price']

In [61]:
y

0       170.0
1       235.0
2        65.0
3        65.0
4       675.0
5       255.0
6       139.0
7       135.0
8       265.0
9       120.0
10      218.0
11      177.0
12      194.0
13      139.0
14       85.0
15       85.0
16       79.0
17      135.0
18      215.0
19      450.0
20      107.0
21      110.0
22      198.0
23      150.0
24       65.0
25      225.0
26       95.0
27      155.0
28      178.0
29      199.0
        ...  
6372     70.0
6373    275.0
6376    200.0
6377    166.0
6381     99.0
6382     89.0
6406    108.0
6407    180.0
6422    185.0
6425    334.0
6426    175.0
6448     89.0
6449    150.0
6453     90.0
6454     90.0
6455    499.0
6464    499.0
6466    250.0
6468    150.0
6469    160.0
6484    150.0
6485    129.0
6488    217.0
6491    300.0
6495    450.0
6507     95.0
6540    256.0
6542     45.0
6564    179.0
6615     74.0
Name: price, Length: 4685, dtype: float64

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [63]:
X.dtypes

host_listings_count                                   int64
accommodates                                          int64
bathrooms                                           float64
bedrooms                                            float64
beds                                                float64
security_deposit                                      int64
guests_included                                       int64
minimum_nights                                        int64
maximum_nights                                        int64
availability_30                                       int64
availability_60                                       int64
availability_365                                      int64
number_of_reviews                                     int64
review_scores_rating                                float64
review_scores_accuracy                              float64
review_scores_cleanliness                           float64
review_scores_checkin                   

In [64]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [65]:
lr = linear_model.LinearRegression()
lr.fit(X_train_trans, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [66]:
lr.score(X_test_trans, y_test)

-1.1513593716881005e+25

In [67]:
y_predict = lr.predict(X_test_trans)

In [68]:
r2_score(y_predict, y_test)

-0.0014188266387391568

In [69]:
mean_squared_error(y_predict, y_test)

4.5967674584691506e+29

In [70]:
model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

/Users/xzhou/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/xzhou/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/xzhou/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/xzhou/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     5.617
Date:                Thu, 13 Sep 2018   Prob (F-statistic):          2.33e-133
Time:                        11:26:33   Log-Likelihood:                -22482.
No. Observations:                3279   AIC:                         4.552e+04
Df Residuals:                    3001   BIC:                         4.722e+04
Df Model:                         277                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        202.5261      4.196     48.264      0.000     194.298     210.754
x1          -116.5558     91.868     -1.269      0.205    -296.686      63.575
x2            20.3928      9.348      2.182      0.029       2.064      38.722
x3            35.0034      5.595      6.256      0.000      24.033      45.974
x4            72.5444      8.371      8.666      0.000      56.132      88.957
x5            48.0889      7.998      6.013      0.000      32.407      63.771
x6             9.1559      4.924      1.859      0.063      -0.499      18.811
x7           -22.9932      5.656     -4.065      0.000     -34.083     -11.903
x8           -13.9111      4.928     -2.823      0.005     -23.573      -4.249
x9             0.1308      4.240      0.031      0.975      -8.182       8.444
x10            8.2991      9.686      0.857      0.392     -10.692      27.290
x11            1.5829     10.328      0.153      0.878     -18.667      21.833
x12           -0.5630      5.743     -0.098      0.922     -11.823      10.697
x13           -7.6774      8.133     -0.944      0.345     -23.625       8.270
x14           -3.3465      8.511     -0.393      0.694     -20.034      13.341
x15            3.3564      6.876      0.488      0.625     -10.126      16.838
x16           16.5412      6.102      2.711      0.007       4.577      28.505
x17            4.0688      6.017      0.676      0.499      -7.728      15.866
x18            0.0890      6.421      0.014      0.989     -12.502      12.680
x19            7.3715      6.164      1.196      0.232      -4.716      19.458
x20           -2.6558      6.578     -0.404      0.686     -15.553      10.241
x21           25.8243     28.831      0.896      0.370     -30.706      82.354
x22           -6.9557      7.679     -0.906      0.365     -22.013       8.101
x23            8.8348      6.257      1.412      0.158      -3.433      21.102
x24           -7.7160      7.416     -1.040      0.298     -22.257       6.825
x25           -6.8712      5.113     -1.344      0.179     -16.897       3.155
x26            2.6368      4.876      0.541      0.589      -6.924      12.198
x27           -0.9074      4.471     -0.203      0.839      -9.675       7.860
x28         7.744e-12   5.75e-12      1.346      0.178   -3.53e-12     1.9e-11
x29            0.2346      4.279      0.055      0.956      -8.156       8.625
x30          -20.4551     14.464     -1.414      0.157     -48.816       7.906
x31           -3.3834      4.417     -0.766      0.444     -12.044       5.277
x32            0.8002      4.294      0.186      0.852      -7.620       9.220
x33          -18.8433     14.466     -1.303      0.193     -47.208       9.521
x34            1.0553      4.275      0.247      0.805      -7.326       9.437
x35           -1.9652      4.280     -0.459      0.646     -10.357       6.427
x3

In [71]:
df_listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,2.018080e+13,2018-08-06,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,none,*Quiet cul de sac in friendly neighborhood *St...,...,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,5858,https://www.airbnb.com/rooms/5858,2.018080e+13,2018-08-06,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,7918,https://www.airbnb.com/rooms/7918,2.018080e+13,2018-08-06,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,none,"Shopping old town, restaurants, McDonald, Whol...",...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16
3,8142,https://www.airbnb.com/rooms/8142,2.018080e+13,2018-08-06,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,none,NaN,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.15
4,8339,https://www.airbnb.com/rooms/8339,2.018080e+13,2018-08-06,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,none,NaN,...,t,STR-0000264,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,t,t,2,0.24


In [72]:
X_train_trans

array([[-0.16435366,  0.39910989, -0.4199021 , ..., -0.03494816,
         0.25076867,  0.29223652],
       [-0.18025909, -0.64200493, -0.4199021 , ..., -0.03494816,
         0.25076867,  0.29223652],
       [-0.17230637,  0.39910989, -0.4199021 , ..., -0.03494816,
         0.25076867,  0.29223652],
       ...,
       [-0.00529938, -1.16256234,  4.79794342, ..., -0.03494816,
         0.25076867,  0.29223652],
       [-0.18025909, -0.64200493, -0.4199021 , ..., -0.03494816,
         0.25076867,  0.29223652],
       [-0.16435366,  0.39910989, -0.4199021 , ..., -0.03494816,
         0.25076867,  0.29223652]])

In [73]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=500, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train_trans, y_train)
y_train_pred = forest.predict(X_train_trans)
y_test_pred = forest.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 8962.591, test: 35519.512
R^2 train: 0.888, test: 0.110
